In [ ]:
import pandas as pd
df = pd.read_csv('Lega_PerSum.csv')
df.head()

In [ ]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to('cuda')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

In [ ]:
import numpy as np
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

premise = df["Judgement"].to_list()
hypothesis = df["Perspective-based Summary"].to_list()

In [ ]:
from tqdm import tqdm

# Initialize the dictionary
NLI_Judgement_Summary = {
    "Judgement": [],
    "Perspective-based Summary": [],
    "Contradiction": [],
    "Neutral": [],
    "Entailment": [],
}

# Loop through the premise (judgments) and hypothesis (summaries) with tqdm for progress tracking
for i, (p, h) in tqdm(enumerate(zip(premise, hypothesis)), total=len(premise)):
    # Encode the premise and hypothesis for NLI
    x = tokenizer.encode(p, h, return_tensors='pt', truncation=True)

    # Get the logits from the model and calculate softmax probabilities
    logits = nli_model(x.to(device))[0]
    probs = logits.softmax(dim=1)

    # Extract probabilities for each class: contradiction, neutral, entailment
    prob_contradiction = probs[:, 0].item()
    prob_neutral = probs[:, 1].item()
    prob_entailment = probs[:, 2].item()

    # Append values to the dictionary
    NLI_Judgement_Summary["Judgement"].append(p)
    NLI_Judgement_Summary["Perspective-based Summary"].append(h)
    NLI_Judgement_Summary["Contradiction"].append(prob_contradiction)
    NLI_Judgement_Summary["Neutral"].append(prob_neutral)
    NLI_Judgement_Summary["Entailment"].append(prob_entailment)

# Now the NLI_Judgement_Summary dictionary will contain the judgments, summaries, and their corresponding NLI scores

In [ ]:
df = pd.DataFrame(NLI_Judgement_Summary)

In [ ]:
entailment_count = (df['Entailment'] > 0.60).sum()
contradiction_count = (df['Contradiction'] < 0).sum()
neutral_count = ((df['Neutral'] > 0.3) & (df["Neutral"] < 0.6)).sum()

# Printing the NLI Scores
print(entailment_count/len(df)), print(contradiction_count/len(df)), print(neutral_count / len(df))